In [1]:
using DifferentialEquations
using Plots; gr()

Plots.GRBackend()

In [2]:
function msd_sys(du,u,p,t)
  # Manual implementation of state transition matrix
  du[1] = u[2]
  du[2] = -p[1]*u[1] - p[2]*u[2]
  
  # Add acceleration due to gravity
  du[2] += p[3]
    
  # Add disturbance in the force
  du[2] +=  p[4]*sin(p[5]*t)
    
  # Add control input
  du[2] += p[6]
end

msd_sys (generic function with 1 method)

In [3]:
g = 9.8 # gravity in metres per second per second
mass = 0.1
k_s = 2.5 # spring constant
k_d = 0.1 # damping coefficient

u0 = [-1., 0.]

f_smc = 10.0

p_simple = [k_s/mass, k_d/mass, g, 0, 0, 0]
p_dist = [k_s/mass, k_d/mass, g, 5., 10., 0]
p_smc = [k_s/mass, k_d/mass, g, 5., 10., 2.]

tspan = (0.0,10.0)

(0.0, 10.0)

In [4]:
function smc_affect!(msd_ss_model)
    sigma = 20.0*msd_ss_model.u[1] + 1.0*msd_ss_model.u[2]
    if sigma < 0
        msd_ss_model.p[6] = +f_smc/mass
    else
        msd_ss_model.p[6] = -f_smc/mass
    end
end
pcb = PeriodicCallback(smc_affect!, 10/10000)

DiffEqBase.DiscreteCallback{DiffEqCallbacks.##40#43{Base.RefValue{Float64}},DiffEqCallbacks.##41#44{#smc_affect!,Float64,Base.RefValue{Float64}},DiffEqCallbacks.##42#45{DiffEqBase.#INITIALIZE_DEFAULT,Bool,Float64,Base.RefValue{Float64},DiffEqCallbacks.##41#44{#smc_affect!,Float64,Base.RefValue{Float64}}}}(DiffEqCallbacks.#40, DiffEqCallbacks.#41, DiffEqCallbacks.#42, (true, true))

In [5]:
prob_simple = ODEProblem(msd_sys,u0,tspan,p_simple)
prob_dist = ODEProblem(msd_sys,u0,tspan,p_dist)
prob_smc = ODEProblem(msd_sys,u0,tspan,p_smc)

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 10.0)
u0: [-1.0, 0.0]

In [6]:
raw_sol = solve(prob_simple);
dist_sol = solve(prob_dist);
smc_sol = solve(prob_smc,callback=pcb);

In [7]:
plot(raw_sol,vars=(0,1),label="Raw")
plot!(dist_sol,vars=(0,1),label="Disturbed")
plot!(smc_sol,vars=(0,1),label="SMC")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 t 
 
<polyline clip-path="url(#clip8302)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.502,360.121 40.0434,359.87 40.5847,359.122 41.1261,357.884 41.6674,356.164 42.2088,353.971 42.7501,351.314 43.2915,348.205 43.8328,344.657 44.3742,340.683 
 44.9156,336.296 45.4569,331.512 45.9983,326.347 46.5396,320.817 47.081,314.94 47.6223,308.734 48.1637,302.218 48.705,295.41 49.2464,288.331 49.7877,281.001 
 50.3291,273.442 50.8705,265.673 51.4118,257.717 51.9532,249.595 52.4945,241.329 53.0359,232.941 53.5772,224.453 54.1186,215.888 54.6599,207.267 55.2013,198.613 
 55.7426,189.948 56.284,181.292 56.8254,172.668 57.3667,164.097 57.9081,155.599 58.4494,147.195 58.9908,138.905 59.5321,130.749 60.0735,122.746 60.6148,114.914 
 61.1562,107.27 61.6975,99.833 62.2389,92.618 62.7802,85.6411 63.3216,78.9172 63.863,72.4604 64.4043,66.2844 64.9457,60.4021 65.487,54.8252 66.0284,49.5648 
 66.5697,44.6307 67.1111,40.0319 67.6524,35.7764 68.1938,31.8713 68.7351,28.3227 69.2765,25.1357 69.8179,22.3145 70.3592,19.8627 70.9006,17.7829 71.4419,16.0767 
 71.9833,14.7448 72.5246,13.7869 73.066,13.2017 73.6073,12.9869 74.1487,13.1394 74.69,13.655 75.2314,14.5284 75.7728,15.7536 76.3141,17.3242 76.8555,19.233 
 77.3968,21.4719 77.9382,24.0321 78.4795,26.904 79.0209,30.0775 79.5622,33.5418 80.1036,37.2855 80.6449,41.2965 81.1863,45.5621 81.7277,50.0687 82.269,54.8025 
 82.8104,59.7497 83.3517,64.8961 83.8931,70.2264 84.4344,75.7256 84.9758,81.3781 85.5171,87.1685 86.0585,93.0811 86.5998,99.1003 87.1412,105.21 87.6825,111.394 
 88.2239,117.637 88.7653,123.922 89.3066,130.232 89.848,136.552 90.3893,142.866 90.9307,149.159 91.472,155.414 92.0134,161.617 92.5547,167.752 93.0961,173.805 
 93.6374,179.761 94.1788,185.607 94.7202,191.329 95.2615,196.915 95.8029,202.351 96.3442,207.626 96.8856,212.729 97.4269,217.648 97.9683,222.373 98.5096,226.895 
 99.051,231.204 99.5923,235.291 100.134,239.148 100.675,242.767 101.216,246.142 101.758,249.267 102.299,252.136 102.84,254.746 103.382,257.091 103.923,259.169 
 104.465,260.978 105.006,262.516 105.547,263.783 106.089,264.777 106.63,265.499 107.171,265.95 107.713,266.13 108.254,266.043 108.795,265.691 109.337,265.077 
 109.878,264.205 110.419,263.08 110.961,261.707 111.502,260.092 112.043,258.243 112.585,256.166 113.126,253.868 113.668,251.359 114.209,248.646 114.75,245.738 
 115.292,242.643 115.833,239.372 116.374,235.934 116.916,232.34 117.457,228.599 117.998,224.723 118.54,220.723 119.081,216.608 119.622,212.392 120.164,208.086 
 120.705,203.702 121.247,199.251 121.788,194.745 122.329,190.195 122.871,185.614 123.412,181.014 123.953,176.406 124.495,171.801 125.036,167.212 125.577,162.649 
 126.119,158.123 126.66,153.645 127.201,149.226 127.743,144.876 128.284,140.605 128.825,136.424 129.367,132.341 129.908,128.366 130.45,124.508 130.991,120.774 
 131.532,117.174 132.074,113.715 132.615,110.405 133.156,107.25 133.698,104.257 134.239,101.432 134.78,98.7795 135.322,96.3047 135.863,94.0119 136.404,91.9047 
 136.946,89.9865 137.487,88.26 138.028,86.7274 138.57,85.3903 139.111,84.2503 139.653,83.3085 140.194,82.5655 140.735,82.0214 141.277,81.6757 141.818,81.5275 
 142.359,81.5753 142.901,81.8172 143.442,82.2506 143.983,82.8725 144.525,83.6795 145.066,84.6674 145.607,85.8317 146.149,87.1673 146.69,88.6687 147.232,90.3304 
 147.773,92.1476 148.314,94.114 148.856,96.2234 149.397,98.469 149.938,100.844 150.48,103.341 151.021,105.953 151.562,108.673 152.104,111.492 152.645,114.402 
 153.186,117.396 153.728,120.465 154.269,123.6 154.81,126.793 155.352,130.035 155.893,133.319 156.435,136.634 156.976,139.974 157.517,143.328 158.059,146.688 
 158.6,150.046 159.141,153.394 159.683,156.723 160.224,160.026 160.765,163.294 161.307,166.521 161.848,169.698 162.389,172.817 162.931,